In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
# Load vehicle data
vehicle_data = pd.read_csv('../data/vehicle data/vehicles.csv')

# Load traffic data
traffic_volumes = pd.read_csv('../data/traffic data/Traffic_Volumes.csv')
bottlenecks = pd.read_csv('../data/traffic data/Bottlenecks.csv')

print("Vehicle Data:")
print(vehicle_data.head())

print("Traffic Volumes:")
print(traffic_volumes.head())

print("Bottlenecks:")
print(bottlenecks.head())

/var/folders/g2/6lzxntx104n2w0wc456bn63m0000gn/T/ipykernel_17901/1194673080.py:2: DtypeWarning: Columns (74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  vehicle_data = pd.read_csv('../data/vehicle data/vehicles.csv')


FileNotFoundError: [Errno 2] No such file or directory: '../data/traffic data/Traffic_Volumes.csv'

In [ ]:
# Drop missing values
vehicle_data.dropna(inplace=True)
traffic_volumes.dropna(inplace=True)
bottlenecks.dropna(inplace=True)

# Rename columns for consistency
vehicle_data.rename(columns={
    'Fuel_Consumption_City': 'city_fuel_efficiency',
    'Fuel_Consumption_Hwy': 'highway_fuel_efficiency',
    'CO2_Emissions': 'co2_emissions'
}, inplace=True)

In [ ]:
# Create a combined fuel efficiency feature
vehicle_data['combined_fuel_efficiency'] = (
    vehicle_data['city_fuel_efficiency'] * 0.55 +
    vehicle_data['highway_fuel_efficiency'] * 0.45
)

# Normalize traffic severity in bottlenecks
bottlenecks['normalized_traffic_severity'] = (
    bottlenecks['traffic_severity'] / bottlenecks['traffic_severity'].max()
)


In [ ]:
# Replace 'location' with the actual column that links traffic and vehicle data
merged_data = pd.merge(traffic_volumes, vehicle_data, on='location', how='inner')

# Add normalized traffic severity
merged_data = pd.merge(merged_data, bottlenecks[['location', 'normalized_traffic_severity']], on='location', how='left')


In [ ]:
X = merged_data[['distance', 'normalized_traffic_severity', 'combined_fuel_efficiency']]
y_fuel = merged_data['fuel_consumption']  # Target: Fuel consumption
y_co2 = merged_data['co2_emissions']      # Target: CO2 emissions


In [ ]:
# Split data
X_train, X_test, y_fuel_train, y_fuel_test = train_test_split(X, y_fuel, test_size=0.2, random_state=42)
_, _, y_co2_train, y_co2_test = train_test_split(X, y_co2, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1)  # Single output for regression
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mse'])


In [ ]:
history_fuel = model.fit(X_train, y_fuel_train, epochs=50, batch_size=32, validation_split=0.2)
history_co2 = model.fit(X_train, y_co2_train, epochs=50, batch_size=32, validation_split=0.2)


In [ ]:
fuel_mse = model.evaluate(X_test, y_fuel_test, verbose=0)[1]
print(f"Fuel Consumption Model MSE: {fuel_mse}")

co2_mse = model.evaluate(X_test, y_co2_test, verbose=0)[1]
print(f"CO₂ Emissions Model MSE: {co2_mse}")


In [ ]:
# Save the fuel consumption model
model.save('../models/fuel_consumption_model.h5')

# Save the CO₂ emissions model
model.save('../models/co2_emissions_model.h5')
